In [91]:
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from PIL import Image
from time import time
import matplotlib.pyplot as plt

In [92]:
train_data = datasets.CIFAR10(
    root = 'data',
    train = True,
    transform = ToTensor(),
    download = True
)

test_data = datasets.CIFAR10(
    root = 'data',
    train = False,
    transform = ToTensor(),
    download = True,
)

train_data.data = torch.from_numpy(train_data.data).to(dtype=torch.float32).permute(0, 3, 1, 2)
train_data.targets = torch.tensor(train_data.targets)

test_data.data = torch.from_numpy(test_data.data).to(dtype=torch.float32).permute(0, 3, 1, 2)
test_data.targets = torch.tensor(test_data.targets)


Files already downloaded and verified
Files already downloaded and verified


In [93]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def conv(target):
    arr = [0]*10
    arr[target.item()] = 1
    return torch.tensor(arr, dtype=torch.float32)

class CustomDataSet:
    def __init__(self, dataset) -> None:
        self.data = dataset.data/255.0
        self.targets = dataset.targets
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return (self.data[idx], conv(self.targets[idx]))
    
train_data = CustomDataSet(train_data)
train_loader = DataLoader(dataset=train_data, batch_size=32, shuffle=True, num_workers=1)


class CNN(nn.Module):
    
    def __init__(self, lr) -> None:
        super().__init__()
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=256, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1)
        self.l1 = nn.Linear(in_features=2048, out_features=128)
        self.l2 = nn.Linear(in_features=128, out_features=64)
        self.l3 = nn.Linear(in_features=64, out_features=10)
        self.dropout = nn.Dropout(0.5)

        self.lr = lr
        self.optimizer = torch.optim.SGD(params=self.parameters(), lr=lr, momentum=0.9)
        self.loss = nn.CrossEntropyLoss()
    
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = self.pool(torch.relu(self.conv4(x)))
        x = torch.relu(self.l1(x.view(-1, 2048)))
        x = self.dropout(x)
        x = torch.relu(self.l2(x))
        x = self.dropout(x)
        x = self.l3(x)
        return x
    
    def predict(self, y_pred):
        m = 0
        I = 0
        for i in range(len(y_pred[0])):
            if y_pred[0][i] > m:
                I = i
                m = y_pred[0][i]
        return I
    
    def train(self, train_loader, testing_data, epochs):
        testing_data.data, testing_data.targets = (testing_data.data/255.0).to(device), testing_data.targets.to(device)
        for epoch in range(epochs):
            for idx, (data, target) in enumerate(train_loader):
                data, target = data.to(device), target.to(device)
                self.optimizer.zero_grad()
                y_pred = self.forward(data)
                y = target
                loss = self.loss(y_pred, y)
                loss.backward()
                self.optimizer.step()
            correct = 0
            for i in range(len(testing_data.data)):
                y_pred = self.forward(testing_data.data[i])
                y_conv = self.predict(self.forward(testing_data.data[i]))
                y = testing_data.targets[i]
                if y_conv == y:
                    correct += 1
            print(f"epoch {epoch+1}: {correct}/{len(testing_data.data)}")
    
cnn = CNN(lr=0.001)
cnn.to(device)
cnn.train(train_loader=train_loader, testing_data=test_data, epochs = 30)


KeyboardInterrupt: 